In [96]:
# from yf_utils import _2_split_train_val_test, _3_random_slices, _4_lookback_slices
from yf_utils import _5_perf_ranks, _6_grp_tuples_sort_sum
from myUtils import pickle_load

# verbose = False  # True prints more output
verbose = True  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"
fp_df_close_clean = 'df_close_clean'
fp_rtn_results = 'rtn_results'
df_close_clean = pickle_load(path_data_dump, fp_df_close_clean)

In [97]:
# number of days to lookback from last df date for performance evaluation
days_lookbacks = [60, 120]
days_lookbacks = [30]
days_lookbacks.sort()  # sort list of integers in ascending order

# number of the most-common symbols from days_lookbacks' performance rankings to keep
n_top_syms = 20  

syms_start = 0  # slice start of set_most_common_syms
syms_end = 10  #  slice end of set_most_common_syms

In [98]:
# load past mean daily return of top_set_syms verus SPY return 
rtn_results = pickle_load(path_data_dump, fp_rtn_results)
rtn_results

,date,rtn_tss_mean,rtn_tss_std,rtn_SPY,rtn_dif
0,2023-02-16,-0.001084,0.033205,-0.013769,0.012684
1,2023-02-17,-0.003160,0.006869,-0.002498,-0.000662
2,2023-02-22,0.006447,0.013909,-0.001378,0.007825


In [99]:
# grp_top_set_syms_n_freq = []  # list of lists of top_set_symbols_n_freq, there are n_samples lists in list
# grp_top_set_syms = []  # list of lists of top_set_symbols, there are n_samples lists in list
grp_most_common_syms = []  # accumulates the best performing symbols from each days_lookback
grp_perf_ranks = []  # accumulates the best performance ranking symbols from each days_lookback 
grp_dates = []  # dates of days_lookbacks 

for d_lookback in days_lookbacks:
  df = df_close_clean.iloc[-d_lookback:]
  perf_ranks, most_common_syms = _5_perf_ranks(df, n_top_syms=n_top_syms)  # get the best performance symbols and rankings
  grp_most_common_syms.append(most_common_syms)
  grp_perf_ranks.append(perf_ranks)  
  grp_dates.append(df.index[0].strftime("%Y-%m-%d")) 

grp_dates.append(df.index[-1].strftime("%Y-%m-%d"))  # get df's last date
grp_dates.sort()

In [100]:
# flatten list of lists of (symbol:frequency)
flat_grp_most_common_syms = [val for sublist in grp_most_common_syms for val in sublist]
# group symbols from set of days_lookbacks (i.e. lb_slices) and sum frequency of the symbols
set_most_common_syms = _6_grp_tuples_sort_sum(flat_grp_most_common_syms, reverse=True)
# get the top few most-frequent symbol:frequency pairs
top_set_syms_n_freq = set_most_common_syms[0:n_top_syms]
# get symbols from top_set_syms_n_freq
top_set_syms = [i[0] for i in top_set_syms_n_freq[syms_start:syms_end]]

In [101]:
print(f'grp_dates: {grp_dates}')
print(f'top_set_syms_n_freq: {top_set_syms_n_freq}')
print(f'top_set_syms: {top_set_syms}')

grp_dates: ['2023-01-10', '2023-02-22']
top_set_syms_n_freq: [('CTLT', 3), ('ELF', 3), ('EVRI', 3), ('NATI', 3), ('OEC', 3), ('TA', 3), ('TSLA', 3), ('WIRE', 3), ('ACLS', 2), ('CIGI', 2), ('CRUS', 2), ('FTSM', 2), ('LRN', 2), ('NVGS', 2), ('PAR', 2), ('PDFS', 2), ('SMG', 2), ('THR', 2), ('TNK', 2), ('ATI', 1)]
top_set_syms: ['CTLT', 'ELF', 'EVRI', 'NATI', 'OEC', 'TA', 'TSLA', 'WIRE', 'ACLS', 'CIGI']


In [102]:
# import webbrowser

# y_quote = 'https://finance.yahoo.com/quote/'
# for sym in top_set_syms:
#   url = y_quote + sym
#   print(url)
#   webbrowser.open(url)  # open symbols in yahoo finance

In [103]:
# # Importing Pandas to create DataFrame
# import pandas as pd

# my_cols = ['date', 'days_lookbacks', 'n_top_syms', 'syms_start', 'syms_end', 'grp_dates', 'top_set_syms_n_freq', 'top_set_syms']
# # Creating Empty DataFrame and Storing it in variable df
# df = pd.DataFrame(columns=my_cols)
# pickle_dump(df, path_data_dump, fp_top_set_syms)
# df

In [104]:
from myUtils import pickle_load, pickle_dump

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"
fp_top_set_syms = 'df_top_set_syms'  # df to store top set symbols

In [105]:
df = pickle_load(path_data_dump, fp_top_set_syms)
df

,date,days_lookbacks,n_top_syms,syms_start,syms_end,grp_dates,top_set_syms_n_freq,top_set_syms
0,2023-02-22,"[60, 120]",20,0,10,"['2022-08-31', '2022-11-25', '2023-02-22']","[('FTSM', 6), ('HZNP', 5), ('ACGL', 4), ('ACLS', 4), ('WYNN', 4), ('ANF', 3), ('ELF', 3), ('FIVE', 3), ('HSBC', 3), ...","['FTSM', 'HZNP', 'ACGL', 'ACLS', 'WYNN', 'ANF', 'ELF', 'FIVE', 'HSBC', 'HTHT']"
1,2023-02-22,[30],20,0,10,"['2023-01-10', '2023-02-22']","[('CTLT', 3), ('ELF', 3), ('EVRI', 3), ('NATI', 3), ('OEC', 3), ('TA', 3), ('TSLA', 3), ('WIRE', 3), ('ACLS', 2), ('...","['CTLT', 'ELF', 'EVRI', 'NATI', 'OEC', 'TA', 'TSLA', 'WIRE', 'ACLS', 'CIGI']"


In [106]:
row_add = [grp_dates[-1], days_lookbacks, n_top_syms, syms_start, syms_end, grp_dates, top_set_syms_n_freq, top_set_syms]
df.loc[len(df)] = row_add
df

,date,days_lookbacks,n_top_syms,syms_start,syms_end,grp_dates,top_set_syms_n_freq,top_set_syms
0,2023-02-22,"[60, 120]",20,0,10,"['2022-08-31', '2022-11-25', '2023-02-22']","[('FTSM', 6), ('HZNP', 5), ('ACGL', 4), ('ACLS', 4), ('WYNN', 4), ('ANF', 3), ('ELF', 3), ('FIVE', 3), ('HSBC', 3), ...","['FTSM', 'HZNP', 'ACGL', 'ACLS', 'WYNN', 'ANF', 'ELF', 'FIVE', 'HSBC', 'HTHT']"
1,2023-02-22,[30],20,0,10,"['2023-01-10', '2023-02-22']","[('CTLT', 3), ('ELF', 3), ('EVRI', 3), ('NATI', 3), ('OEC', 3), ('TA', 3), ('TSLA', 3), ('WIRE', 3), ('ACLS', 2), ('...","['CTLT', 'ELF', 'EVRI', 'NATI', 'OEC', 'TA', 'TSLA', 'WIRE', 'ACLS', 'CIGI']"
2,2023-02-22,[30],20,0,10,"[2023-01-10, 2023-02-22]","[(CTLT, 3), (ELF, 3), (EVRI, 3), (NATI, 3), (OEC, 3), (TA, 3), (TSLA, 3), (WIRE, 3), (ACLS, 2), (CIGI, 2), (CRUS, 2)...","[CTLT, ELF, EVRI, NATI, OEC, TA, TSLA, WIRE, ACLS, CIGI]"


In [107]:
# df
_df = df.astype(str)
# df.loc[df.astype(str).drop_duplicates(subset=['date', 'days_lookbacks'])]
_df = _df.drop_duplicates(subset=['date', 'days_lookbacks'])
_df

,date,days_lookbacks,n_top_syms,syms_start,syms_end,grp_dates,top_set_syms_n_freq,top_set_syms
0,2023-02-22,"[60, 120]",20,0,10,"['2022-08-31', '2022-11-25', '2023-02-22']","[('FTSM', 6), ('HZNP', 5), ('ACGL', 4), ('ACLS', 4), ('WYNN', 4), ('ANF', 3), ('ELF', 3), ('FIVE', 3), ('HSBC', 3), ...","['FTSM', 'HZNP', 'ACGL', 'ACLS', 'WYNN', 'ANF', 'ELF', 'FIVE', 'HSBC', 'HTHT']"
1,2023-02-22,[30],20,0,10,"['2023-01-10', '2023-02-22']","[('CTLT', 3), ('ELF', 3), ('EVRI', 3), ('NATI', 3), ('OEC', 3), ('TA', 3), ('TSLA', 3), ('WIRE', 3), ('ACLS', 2), ('...","['CTLT', 'ELF', 'EVRI', 'NATI', 'OEC', 'TA', 'TSLA', 'WIRE', 'ACLS', 'CIGI']"


In [108]:
pickle_dump(_df, path_data_dump, fp_top_set_syms)

In [109]:
_df = pickle_load(path_data_dump, fp_top_set_syms)
_df

,date,days_lookbacks,n_top_syms,syms_start,syms_end,grp_dates,top_set_syms_n_freq,top_set_syms
0,2023-02-22,"[60, 120]",20,0,10,"['2022-08-31', '2022-11-25', '2023-02-22']","[('FTSM', 6), ('HZNP', 5), ('ACGL', 4), ('ACLS', 4), ('WYNN', 4), ('ANF', 3), ('ELF', 3), ('FIVE', 3), ('HSBC', 3), ...","['FTSM', 'HZNP', 'ACGL', 'ACLS', 'WYNN', 'ANF', 'ELF', 'FIVE', 'HSBC', 'HTHT']"
1,2023-02-22,[30],20,0,10,"['2023-01-10', '2023-02-22']","[('CTLT', 3), ('ELF', 3), ('EVRI', 3), ('NATI', 3), ('OEC', 3), ('TA', 3), ('TSLA', 3), ('WIRE', 3), ('ACLS', 2), ('...","['CTLT', 'ELF', 'EVRI', 'NATI', 'OEC', 'TA', 'TSLA', 'WIRE', 'ACLS', 'CIGI']"


In [110]:
_df["top_set_syms_n_freq"][1]

"[('CTLT', 3), ('ELF', 3), ('EVRI', 3), ('NATI', 3), ('OEC', 3), ('TA', 3), ('TSLA', 3), ('WIRE', 3), ('ACLS', 2), ('CIGI', 2), ('CRUS', 2), ('FTSM', 2), ('LRN', 2), ('NVGS', 2), ('PAR', 2), ('PDFS', 2), ('SMG', 2), ('THR', 2), ('TNK', 2), ('ATI', 1)]"

In [ ]:
prev_top_set_syms = df["top_set_syms"].iloc[-2]
prev_top_set_syms

In [ ]:
import pandas as pd
# pd.options.display.width = 120
pd.options.display.max_colwidth = 120

print(f'_df["top_set_syms_n_freq"]:\n{_df["top_set_syms_n_freq"]}\n')
l_last_top_set_syms_n_freq = _df['top_set_syms_n_freq'].iloc[-1]

print(f'_df["top_set_syms"]:\n{_df["top_set_syms"]}')
l_last_top_set_syms = _df['top_set_syms'].iloc[-1]
print(f'last row of top_set_syms:\n{l_last_top_set_syms}')

In [ ]:
recent_top_set_syms = []
for symbols in _df["top_set_syms"]:
  recent_top_set_syms.append(symbols)
recent_top_set_syms

# flatten symbols_download_err which is a list-of-lists
recent_top_set_syms = [val for sublist in recent_top_set_syms for val in sublist]
recent_top_set_syms
# get list of unique symbols in df_OHLCV, list(df) is a list of tuples
# e.g.: [('AAPL', 'Open')..('AAPL', 'Volume'),...
#        ('ZZZZ', 'Open')..('ZZZZ', 'Volume')]
recent_top_set_syms = list(set(recent_top_set_syms))
recent_top_set_syms.append("SPY")
recent_top_set_syms.sort()
print(recent_top_set_syms)


In [ ]:
# my_cols.sort() will not change order of l_last_top_set_syms 
my_cols = recent_top_set_syms.copy()  
my_cols.sort()
# my_cols.append('SPY')
_df = df_close_clean[my_cols].tail(10)
# print(_df)
print(_df.pct_change())
_df.pct_change().tail(1)[0:10]

In [ ]:
# today's return from previous top_set_syms
prev_top_set_syms_n_SPY = prev_top_set_syms + ['SPY']
rtn_prev_top_set_syms = _df.pct_change().tail(1)[prev_top_set_syms_n_SPY]
rtn_prev_top_set_syms_mean = rtn_prev_top_set_syms.mean(axis=1)[0]
rtn_prev_top_set_syms_std = rtn_prev_top_set_syms.std(axis=1)[0]
# print(f'rtn_prev_top_set_syms:\n{rtn_prev_top_set_syms}')
print(f'rtn_prev_top_set_syms_mean:                                                {rtn_prev_top_set_syms_mean:>10.6f}')
print(f'rtn_prev_top_set_syms_std:                                                 {rtn_prev_top_set_syms_std:>10.6f}\n')

rtn_SPY = _df.pct_change().tail(1)['SPY'][0]  # [0] changes pd.Series to a value
rtn_dif = rtn_prev_top_set_syms_mean - rtn_SPY
print(f'rtn_SPY:                                                                   {rtn_SPY:>10.6f}\n')                  
print(f"Difference in daily return between yesterday's top_set_syms_mean and SPY:  {rtn_dif:>10.6f}")

print("\nToday's return of previous top_set_syms and SPY")
rtn_prev_top_set_syms

In [ ]:
_date = rtn_prev_top_set_syms.tail(1).index.strftime('%Y-%m-%d')[0]
_rtn_tss_mean = rtn_prev_top_set_syms_mean
_rtn_tss_std = rtn_prev_top_set_syms_std
_rtn_SPY = rtn_SPY
_rtn_dif = rtn_dif

In [ ]:
row_add = [_date, _rtn_tss_mean, _rtn_tss_std, _rtn_SPY, _rtn_dif]

In [ ]:
# my_cols
row_add

In [ ]:
# # # Creating Empty DataFrame and Storing it in variable df
# my_cols = ['date', 'rtn_tss_mean', 'rtn_tss_std', 'rtn_SPY', 'rtn_dif']
# rtn_results = pd.DataFrame(columns=my_cols)

rtn_results.loc[len(rtn_results)] = row_add
rtn_results

In [ ]:
rtn_results = rtn_results.drop_duplicates(subset=['date'])
rtn_results

In [ ]:
pickle_dump(rtn_results, path_data_dump, fp_rtn_results)

In [ ]:
_df = pickle_load(path_data_dump, fp_rtn_results)
_df.sort_values(by='date', ascending=False)